<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/Parapolitka_articles_scraper%2Cteliki_ergasia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [ ]:
def get_webpage_soup(url):

  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)

  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    print(f"{url} was scraped successfully")
    soup = BeautifulSoup(response.text, 'html.parser')
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_article_elements_from_soup(soup, article_url):
    article_block = soup.find("div", class_="articleInner")

    # Title
    try:
        title = article_block.find("h1", class_="articleIntro__title").text.strip()
    except:
        title = ""

    # Datetime
    try:
        datetime_tag = article_block.find("time")
        datetime_str = datetime_tag["datetime"]
        date_str, time_str = datetime_str.split("T")[0], datetime_str.split("T")[1].split("Z")[0]
    except:
        date_str = ""
        time_str = ""

    # Section
    try:
        section = article_block.find("a", href=True).text.strip()
    except:
        section = ""

    # Author
    try:
        author = article_block.find("span", class_="articleAuthor__name").text.strip()
    except:
        author = ""

    # Excerpt
    try:
        excerpt = article_block.find("p", class_="articleIntro__text").text.strip()
    except:
        excerpt = ""

    # Text body – συλλογή όλων των παραγράφων
    try:
        body_div = article_block.find("div", class_="articleInner__body contentBody")
        paragraphs = body_div.find_all("p")
        text_body = "\n".join(p.text.strip() for p in paragraphs if p.text.strip() != "")
    except:
        text_body = ""

    # Feature image link
    try:
        feat_img_link = article_block.find("img")["src"]
    except:
        feat_img_link = ""

    # Feature image caption
    try:
        image_caption = article_block.find("div", class_="img-caption mb-5").text.strip()
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

def scrape_parapolitika_articles_pages(teasers_df):
    all_articles_dataL = []

    for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
        article_url = row['article_url']
        soup = get_webpage_soup(article_url)
        if soup:
            articleD = get_article_elements_from_soup(soup, article_url)
            all_articles_dataL.append(articleD)
            delay = random.uniform(1, 3)
            time.sleep(delay)

    df = pd.DataFrame(all_articles_dataL)
    print(f"\nScraping completed. Fetched {len(df)} articles.")
    return df

In [ ]:
from tqdm import tqdm #μενει ιδιος
def scrape_parapolitika_articles_pages(teasers_df):
  # δημιουργία κενής λίστας στην οποία θα προσθέτουμε ένα-ένα τα λεξικά με τα δεδομένα των άρθρων που θα σκραπάρουμε
  all_articles_dataL = []
  # λούπα σε κάθε row του teasers_df για να παίρνουμε ένα-ένα τα urls των άρθρων και να το σκραπάρουμε
  # χρησιμοποιούμε το tqdm για να βλέπουμε σε ένα progress bar πού βρισκόμαστε και πόσος χρόνος απομένει ακόμα
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    # παίρνουμε το url του άρθρου που θα σκραπάρουμε από τη στήλη "article_url" του row του teasers_df στο οποίο βρίσκεται η for loop
    article_url = row['article_url']
    # scrape της σελίδας του url του κάθε άρθρου με το function που φτιάξαμε στο βήμα 2
    soup = get_webpage_soup(article_url)
    # δημιουργία του λεξικού με τα δεδομένα του κάθε άρθρου με το function που φτιάξαμε στο βήμα 9
    articleD = get_article_elements_from_soup(soup, article_url)
    # προσθήκη του λεξικού με τα δεδομένα του κάθε άρθρου στη λίστα που δημιουργήσαμε στη αρχή
    all_articles_dataL.append(articleD)
    # καθυστέρηση ανάμεσα στα requests στο server
    delay = random.uniform(1, 3)
    time.sleep(delay)
  # δημιουργία dataframe από τη λίστα των λεξικών με τα δεδομένα κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
#δοκιμή του scraper
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
teasers_df=pd.read_csv('/content/drive/MyDrive/CSVFILES/ergatika_atiximataparapolitika_df.csv')
teasers_df.head()

,title,article_url,date,time,author,section,teaser
0,Εργατικό ατύχημα στην Πυλαία,https://www.parapolitika.gr/ellada/article/327...,2018-09-13,08:44:13Z,PARAPOLITIKA NEWSROOM,ΕΛΛΑΔΑ,NaN
1,Εργατικό ατύχημα στην Κνωσό,https://www.parapolitika.gr/ellada/article/304...,2018-03-21,10:40:54Z,PARAPOLITIKA NEWSROOM,ΕΛΛΑΔΑ,NaN
2,Εργατικό ατύχημα στην Ελευσίνα,https://www.parapolitika.gr/ellada/article/275...,2017-08-31,05:01:11Z,PARAPOLITIKA NEWSROOM,ΕΛΛΑΔΑ,NaN
3,Ο Γολγοθάς της σύνταξης μετά το εργατικό ατύχημα,https://www.parapolitika.gr/parapolitika-tv/ar...,2021-11-25,09:55:17Z,NaN,PARAPOLITIKA TV,NaN
4,Εργατικό ατύχημα στα Χανιά με έναν τραυματία,https://www.parapolitika.gr/ellada/article/259...,2017-05-12,04:57:26Z,PARAPOLITIKA NEWSROOM,ΕΛΛΑΔΑ,NaN


In [ ]:
articles_df = scrape_parapolitika_articles_pages(teasers_df)
articles_df

Processing rows:   0%|          | 0/2051 [00:00<?, ?it/s]

https://www.parapolitika.gr/ellada/article/327391/ergatiko-atichima-stin-pilea/ was scraped successfully


Processing rows:   0%|          | 1/2051 [00:03<1:47:50,  3.16s/it]

https://www.parapolitika.gr/ellada/article/304506/ergatiko-atichima-stin-knoso/ was scraped successfully


Processing rows:   0%|          | 2/2051 [00:07<2:02:15,  3.58s/it]

...

Processing rows: 100%|█████████▉| 2050/2051 [1:40:56<00:03,  3.18s/it]

https://www.parapolitika.gr/media/article/1288867/thauma-zois-padreutike-27hronos-tetrapligikos-piasei-stauro-theofania-2021/ was scraped successfully


Processing rows: 100%|██████████| 2051/2051 [1:41:00<00:00,  2.95s/it]


Scraping completed. Fetshed 2051 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.parapolitika.gr/ellada/article/327...,Εργατικό ατύχημα στην Πυλαία,2018-09-13,08:41:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,Σε κρίσιμη κατάσταση νοσηλεύεται 63χρονος,"Μάχη για τη ζωή του δίνει ένας 63χρονος, ο οπο...",https://s.parapolitika.gr/images/1130x667/jpg/...,
1,https://www.parapolitika.gr/ellada/article/304...,Εργατικό ατύχημα στην Κνωσό,2018-03-21,10:39:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,Ο εργάτης υπό αδιευκρίνιστες συνθήκες έπεσε απ...,Σε σοβαρή κατάσταση νοσηλεύεται στο Βενιζέλειο...,https://s.parapolitika.gr/images/1130x667/jpg/...,
2,https://www.parapolitika.gr/ellada/article/275...,Εργατικό ατύχημα στην Ελευσίνα,2017-08-31,04:58:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,Διενεργείται προανάκριση από το Κεντρικό Λιμεν...,Λιμενεργάτης τραυματίστηκε στο μέτωπο από αλυσ...,https://s.parapolitika.gr/images/1130x667/jpg/...,
3,https://www.parapolitika.gr/parapolitika-tv/ar...,Ο Γολγοθάς της σύνταξης μετά το εργατικό ατύχημα,2021-11-25,11:55:17,,,,,,
4,https://www.parapolitika.gr/ellada/article/259...,Εργατικό ατύχημα στα Χανιά με έναν τραυματία,2017-05-12,04:52:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,Τις συνθήκες του εργατικού ατυχήματος ερευνούν...,Εργατικό ατύχημα σημειώθηκε σε πολυκατοικία επ...,https://s.parapolitika.gr/images/1130x667/jpg/...,
...,...,...,...,...,...,...,...,...,...,...
2046,https://www.parapolitika.gr/media/article/1198...,Τράπερ «παλικάρια» έγιναν μαλλιά κουβάρια - Η ...,2022-06-24,04:13:44,MEDIA,Parapolitika Newsroom,Πλούσιο ρεπορτάζ στην On Time,,https://s.parapolitika.gr/images/1130x667/jpg/...,
2047,https://www.parapolitika.gr/ellada/article/135...,Αχαρνές: “Έχουν περάσει 2 μήνες και δεν έχουν ...,2024-03-06,17:34:44,ΕΛΛΑΔΑ,Parapolitika Newsroom,Πώς αποφεύχθηκε το παραλίγο νέο ατύχημα με σύγ...,Λειτούργησε το σύστημα της τηλεδιοίκησης και...,https://s.parapolitika.gr/images/1130x667/jpg/...,
2048,https://www.parapolitika.gr/ellada/article/130...,Αναστάτωση στο λιμάνι του Ηρακλείου: Διαμαρτυρ...,2023-09-06,19:32:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,Kάποιοι από τους διαδηλωτές πέταξαν μπογιές στ...,Αναστάτωση επικράτησε νωρίς το βράδυ της Τετάρ...,https://s.parapolitika.gr/images/1130x667/jpg/...,
2049,https://www.parapolitika.gr/ellada/article/125...,"Λύτρας στα Παραπολιτικά 90,1 για τραγωδία στα ...",2023-03-02,08:52:00,ΕΛΛΑΔΑ,Parapolitika Newsroom,«Δεν γίνεται σε ένα ευρωπαϊκό κράτος να έχουμε...,,https://s.parapolitika.gr/images/1130x667/jpg/...,


In [ ]:
articles_df.to_csv('/content/drive/MyDrive/Colab Notebooks/ergatika_atiximatadf_articles.csv', index=False)
articles_df.to_csv('/content/drive/MyDrive/CSVFILES/ergatika_atiximatadf_articles.csv', index=False, encoding='utf-8')

In [ ]:
articles_df.to_csv('/content/drive/MyDrive/CSVFILES/ergatika_atiximataparapolitika_articles.csv', index=False)

In [ ]:
#df.to_csv('/content/drive/MyDrive/CSVFILES/parapolitika_articles.csv', index=False)

NameError: name 'df' is not defined